In [4]:
# -----------------------------------------------------------------------------
#  Proyecto: R.O.V.E.R.
#  Módulo: Sistema de Rastreo Solar (Simulación)
#  Descripción:
#      Este script implementa la simulación completa del panel de rastreo solar
#      utilizado en el sistema R.O.V.E.R., responsable de orientar el panel
#      fotovoltaico hacia la mayor incidencia lumínica disponible.
#
#      El panel incluye dos servomotores (horizontal y vertical) y cuatro
#      sensores LDR distribuidos en la estructura para obtener mediciones de
#      luminosidad en tiempo real. Debido a que el proyecto original utiliza
#      Arduino, esta versión en Python se emplea para simular el algoritmo de
#      seguimiento solar y evaluar su desempeño sin hardware físico.
#
#  Equipo de desarrollo:
#      • Carabeo López Darwin Alberto
#      • Hernández Mendoza Abigail
#      • Hernández Tlapa Emmanuel
#      • Lara Hernández Daniel
#      • Villalva Cervantes Jovany
#
#  Notas:
#      El código está orientado a la evaluación del comportamiento dinámico
#      del sistema, simulando el funcionamiento del panel de alimentación solar
#      del R.O.V.E.R. mediante Programación Orientada a Objetos (POO).
# -----------------------------------------------------------------------------

import random
import time


# =============================================================================
#  Clase ServoController
# =============================================================================
class ServoController:
    """
    Clase que representa un servomotor dentro de la simulación.
    Controla límites, posición actual y desplazamientos incrementales.
    """

    def __init__(self, name, initial_angle, limit_low, limit_high):
        """
        Constructor del servomotor.

        Parámetros
        ----------
        name : str
            Nombre identificador del servomotor.
        initial_angle : int
            Ángulo inicial al arrancar el sistema.
        limit_low : int
            Límite inferior permitido.
        limit_high : int
            Límite superior permitido.
        """
        self.name = name
        self.angle = initial_angle
        self.limit_low = limit_low
        self.limit_high = limit_high

    def begin(self):
        """Imprime la posición inicial del servomotor."""
        print(f"{self.name} iniciado en {self.angle}°")

    def set_angle(self, new_angle):
        """
        Asigna un nuevo ángulo al servomotor asegurando que
        permanezca dentro de los límites definidos.
        """
        if new_angle < self.limit_low:
            new_angle = self.limit_low
        if new_angle > self.limit_high:
            new_angle = self.limit_high
        
        self.angle = new_angle
        print(f"{self.name} movido a {self.angle}°")

    def increment(self, value):
        """
        Cambia el ángulo de forma incremental.

        value > 0  → mueve en sentido positivo  
        value < 0  → mueve en sentido negativo
        """
        self.set_angle(self.angle + value)

    def get_angle(self):
        """Retorna el ángulo actual del servomotor."""
        return self.angle


# =============================================================================
#  Clase LDR (Simulación de Sensor de Luz)
# =============================================================================
class LDR:
    """
    Simula un sensor LDR físico.  
    Devuelve valores aleatorios dentro de un rango razonable para representar
    condiciones de iluminación variables.
    """

    def __init__(self, name):
        self.name = name

    def read(self):
        """
        Simula la lectura de un sensor LDR.

        Retorna un valor entre 100 y 900 para modelar escenarios realistas.
        """
        value = random.randint(100, 900)
        print(f"{self.name}: {value}")
        return value


# =============================================================================
#  Clase SolarTracker (Control Principal del Sistema)
# =============================================================================
class SolarTracker:
    """
    Clase central que controla todo el sistema de rastreo solar.  
    Lee sensores, calcula promedios, ajusta tolerancias y mueve los servos para
    orientar el panel hacia la dirección de mayor iluminación.
    """

    def __init__(self):
        # --- Servomotores ---
        self.horizontal = ServoController("Horizontal", 90, 0, 180)
        self.vertical   = ServoController("Vertical",   120, 90, 180)

        # --- Sensores LDR ---
        self.ldr_lt = LDR("LDR Superior Izquierda")
        self.ldr_rt = LDR("LDR Superior Derecha")
        self.ldr_ld = LDR("LDR Inferior Izquierda")
        self.ldr_rd = LDR("LDR Inferior Derecha")

        # Parámetros dinámicos (tolerancia y tiempo)
        self.tol = 100
        self.dtime = 100  # milisegundos

    def begin(self):
        """Inicializa el sistema de rastreo."""
        print("\n=== Iniciando sistema de rastreo solar ===")
        self.horizontal.begin()
        self.vertical.begin()
        print("===========================================\n")

    def update(self):
        """Ejecuta un ciclo completo de lectura y ajuste del panel."""

        # --- Lectura de LDR ---
        lt = self.ldr_lt.read()
        rt = self.ldr_rt.read()
        ld = self.ldr_ld.read()
        rd = self.ldr_rd.read()

        # --- Promedios de zonas ---
        avt = (lt + rt) // 2  # Superior
        avd = (ld + rd) // 2  # Inferior
        avl = (lt + ld) // 2  # Izquierda
        avr = (rt + rd) // 2  # Derecha

        # Promedio global (simula intensidad ambiental)
        veg = (avt + avd + avl + avr) // 4
        print(f"veg={veg}")

        # --- Ajuste dinámico de parámetros ---
        if 0 < veg < 300:
            # Simulación del mapeo del Arduino (map())
            self.tol = int((veg - 10) * (100 - 5) / (300 - 10) + 5)
            self.dtime = int((veg - 10) * (50 - 100) / (300 - 10) + 100)
        else:
            self.tol = 50
            self.dtime = 50

        print(f"tol={self.tol}, dtime={self.dtime}")

        # --- Diferencias entre zonas ---
        dvert = avt - avd
        dhoriz = avl - avr

        # --- Control del eje vertical ---
        if abs(dvert) > self.tol:
            if avt > avd:
                self.vertical.increment(+1)
            else:
                self.vertical.increment(-1)

        # --- Control del eje horizontal ---
        if abs(dhoriz) > self.tol:
            if avl < avr:
                self.horizontal.increment(-1)
            else:
                self.horizontal.increment(+1)

        print("\n-----------------------------------\n")
        time.sleep(self.dtime / 1000)  # Conversión a segundos


In [ ]:
# -----------------------------------------------------
#  Ejecución del sistema
# -----------------------------------------------------
tracker = SolarTracker()
tracker.begin()

while True:
    tracker.update()
    


=== Iniciando sistema de rastreo solar ===
Horizontal iniciado en 90°
Vertical iniciado en 120°

LDR Superior Izquierda: 186
LDR Superior Derecha: 641
LDR Inferior Izquierda: 776
LDR Inferior Derecha: 631
veg=558
tol=50, dtime=50
Vertical movido a 119°
Horizontal movido a 89°

-----------------------------------

LDR Superior Izquierda: 309
LDR Superior Derecha: 529
LDR Inferior Izquierda: 532
LDR Inferior Derecha: 624
veg=498
tol=50, dtime=50
Vertical movido a 118°
Horizontal movido a 88°

-----------------------------------

LDR Superior Izquierda: 440
LDR Superior Derecha: 717
LDR Inferior Izquierda: 301
LDR Inferior Derecha: 248
veg=426
tol=50, dtime=50
Vertical movido a 119°
Horizontal movido a 87°

-----------------------------------

LDR Superior Izquierda: 435
LDR Superior Derecha: 796
LDR Inferior Izquierda: 757
LDR Inferior Derecha: 655
veg=660
tol=50, dtime=50
Vertical movido a 118°
Horizontal movido a 86°

-----------------------------------

LDR Superior Izquierda: 659
LD